# Setup

In [28]:
import pandas as pd
from pathlib import Path

In [29]:
#input args
log_name = 'logs/LOG_108.csv'
ticks_per_second_mb = 1000
out_dir = './out'

#derived args
split_logs_dir = f'{out_dir}/split_logs/'

In [30]:
#create output folders
Path(f'{out_dir}/split_logs').mkdir(parents=True, exist_ok=True)
Path(f'{out_dir}/plots').mkdir(parents=True, exist_ok=True)

# Full Log

In [31]:
full_log = pd.read_csv(log_name, delimiter = ';')
full_log.index.rename('Log Entry', inplace = True)
full_log.head()

,Timestamp,Log Entry Type,Data
Log Entry,,,
0,501,3,"0,0,0"
1,501,4,"0,0,0"
2,511,1,"1,1,0,0,0"
3,511,1,"1,2,0,0,0,0,0,0,0"
4,504,2,1


In [32]:
#convert timestamp ticks to seconds
full_log['Time_s'] = full_log['Timestamp']/ticks_per_second_mb

In [33]:
#change entry type ids to enums
full_log['Log Entry Type'].replace({1:'SENSOR', 2:'STATE', 3:'ESTIMATOR_VAR', 4: 'CONTROLLER_OUTPUT', 5:'MOTOR', 6:'MSG'}, inplace = True)

# Sensor Log

In [34]:
#extract sensor entries
sensor_log = full_log[full_log['Log Entry Type'] == 'SENSOR'].copy()
#extract data for sensor entries
sensor_log[['Sensor Board ID', 'Sensor Type', 'Data']] = sensor_log['Data'].str.split(',', expand = True, n = 2)

#change sensor type ids to enums
sensor_log['Sensor Type'] = sensor_log['Sensor Type'].astype(int)
sensor_log['Sensor Type'].replace({1:'BARO', 2:'IMU', 3:'GPS', 4: 'BATTERY'}, inplace = True)

In [35]:
#extract individual sensor types
baro_log = sensor_log[sensor_log['Sensor Type'] == 'BARO'].copy()
imu_log = sensor_log[sensor_log['Sensor Type'] == 'IMU'].copy()
gps_log = sensor_log[sensor_log['Sensor Type'] == 'GPS'].copy()
battery_log = sensor_log[sensor_log['Sensor Type'] == 'BATTERY'].copy()

## Baro Log

In [36]:
#extract data for baro sensors
baro_log[['Pressure', 'Temperature', 'Sensor Board Timestamp']] = baro_log['Data'].str.split(',', expand = True)
baro_log.drop(columns = ['Data'], inplace = True)
#reorder columns
baro_log = baro_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type', 'Pressure', 'Temperature']]

#convert temperature to Celsius
fmt_tmp = lambda x: x / 100
baro_log['Temperature'] = baro_log['Temperature'].apply(pd.to_numeric).apply(fmt_tmp)
baro_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Pressure,Temperature
Log Entry,,,,,,,,
2,511,0.511,SENSOR,1,0,BARO,0,0.00
6,511,0.511,SENSOR,2,0,BARO,0,0.00
8,511,0.511,SENSOR,3,0,BARO,0,0.00
17,521,0.521,SENSOR,1,18083,BARO,95489,29.72
19,521,0.521,SENSOR,2,18013,BARO,95466,29.47


In [37]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
baro_log.to_csv(f'{split_logs_dir}/baro.csv')

## IMU Log

In [38]:
#generate Acc_x, Acc_y, Acc_z, Gyro_x, Gyro_y, Gyro_z
dim3 = lambda x : [x + '_' + s for s in ['x', 'y', 'z']]
imu_cols = dim3('Acc') + dim3('Gyro')

#extract data for imu sensors
imu_log[imu_cols + ['Sensor Board Timestamp']] = imu_log['Data'].str.split(',', expand = True)
imu_log.drop(columns = ['Data'], inplace = True)
#reorder columns
imu_log = imu_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type'] + imu_cols]

#scale acceleration and rotation to float

#convert to gs
fmt_acc = lambda x: x / 1024
#convert to degrees per second
fmt_gyro = lambda x: x / 65.5

imu_log[dim3('Acc')] = imu_log[dim3('Acc')].apply(pd.to_numeric).apply(fmt_acc)
imu_log[dim3('Gyro')] = imu_log[dim3('Gyro')].apply(pd.to_numeric).apply(fmt_gyro)

imu_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z
Log Entry,,,,,,,,,,,,
3,511,0.511,SENSOR,1,0,IMU,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,511,0.511,SENSOR,2,0,IMU,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,511,0.511,SENSOR,3,0,IMU,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18,521,0.521,SENSOR,1,18074,IMU,-0.045898,0.010742,-1.026367,0.137405,-0.152672,-0.259542
20,521,0.521,SENSOR,2,18004,IMU,0.001953,-0.024414,-0.995117,0.015267,-0.229008,-0.137405


In [39]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
imu_log.to_csv(f'{split_logs_dir}/imu.csv')

## GPS Log

In [40]:
gps_cols = ['Hour', 'Minute', 'Second', 'Latitude [deg]', 'Latitude [decimal]', 'Longitude [deg]', 'Longitude [decimal]', 'Satellite', 'Altitude', 'HDOP']
gps_log[gps_cols] = gps_log['Data'].str.split(',', expand = True)
gps_log = gps_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + gps_cols]
gps_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Hour,Minute,Second,Latitude [deg],Latitude [decimal],Longitude [deg],Longitude [decimal],Satellite,Altitude,HDOP
Log Entry,,,,,,,,,,,,,,,
5,504,0.504,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
10,504,0.504,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0
11,508,0.508,SENSOR,3,GPS,0,0,0,0,0,0,0,0,0,0
12,508,0.508,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
13,512,0.512,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0


In [41]:
gps_log.to_csv(f'{split_logs_dir}/gps.csv')

## Battery Log

In [42]:
battery_cols = ['Battery', 'Consumption', 'Current', 'Supply']
battery_log[battery_cols] = battery_log['Data'].str.split(',', expand = True)
battery_log = battery_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + battery_cols]
battery_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Battery,Consumption,Current,Supply
Log Entry,,,,,,,,,
471,1078,1.078,SENSOR,1,BATTERY,12456,0,232,3318
1333,2098,2.098,SENSOR,1,BATTERY,12458,0,217,3317
2178,3118,3.118,SENSOR,1,BATTERY,12457,0,234,3319
3049,4138,4.138,SENSOR,1,BATTERY,12455,0,250,3318
3874,5158,5.158,SENSOR,1,BATTERY,12458,0,216,3318


In [43]:
battery_log.to_csv(f'{split_logs_dir}/battery.csv')

# State Log

In [44]:
#IDLE = 1, AIRBRAKE_TEST, THRUSTING, COASTING, DESCENT, RECOVERY

In [45]:
#extract state change entries
state_log = full_log[full_log['Log Entry Type'] == 'STATE'].copy()
state_log.rename(columns={'Data': 'State'}, inplace = True)
state_log['State'] = state_log['State'].astype(int)
state_log['State'].replace({1:'IDLE', 2:'AIRBRAKE_TEST', 3:'THRUSTING', 4:'COASTING', 5:'DESCENT', 6:'RECOVERY'}, inplace = True)
state_log.head()

,Timestamp,Log Entry Type,State,Time_s
Log Entry,,,,
4,504,STATE,IDLE,0.504
95,609,STATE,IDLE,0.609
177,704,STATE,IDLE,0.704
260,804,STATE,IDLE,0.804
343,904,STATE,IDLE,0.904


In [46]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
state_log.to_csv(f'{split_logs_dir}/state_transitions.csv')

# Estimator Variable Log

In [47]:
#extract estimator var entries
estimator_var_log = full_log[full_log['Log Entry Type'] == 'ESTIMATOR_VAR'].copy()
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log['Data'].str.split(',', expand = True)

estimator_var_log.drop(columns = ['Data'], inplace = True)
#reorder columns
estimator_var_log = estimator_var_log[['Timestamp', 'Time_s', 'Log Entry Type', 'World Position (z)', 'Rocket Velocity (z)', 'Rocket Acceleration (z)']]

fmt_est = lambda x: x / 1000
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']].apply(pd.to_numeric).apply(fmt_est)
estimator_var_log.head()

,Timestamp,Time_s,Log Entry Type,World Position (z),Rocket Velocity (z),Rocket Acceleration (z)
Log Entry,,,,,,
0,501,0.501,ESTIMATOR_VAR,0.0,0.000,0.000
15,513,0.513,ESTIMATOR_VAR,0.0,0.000,0.000
23,523,0.523,ESTIMATOR_VAR,0.0,0.000,0.054
31,531,0.531,ESTIMATOR_VAR,0.0,0.001,0.060
39,541,0.541,ESTIMATOR_VAR,0.0,0.001,0.057


In [48]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
estimator_var_log.to_csv(f'{split_logs_dir}/estimator_var.csv')

# Controller Output Log

In [49]:
#extract estimator var entries
controller_out_log = full_log[full_log['Log Entry Type'] == 'CONTROLLER_OUTPUT'].copy()
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log['Data'].str.split(',', expand = True)

controller_out_log.drop(columns = ['Data'], inplace = True)
#reorder columns
controller_out_log = controller_out_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Controller Output', 'Reference Error', 'Integrated Error']]

fmt_cnt = lambda x: x / 1000
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']].apply(pd.to_numeric).apply(fmt_cnt)
controller_out_log.head()

,Timestamp,Time_s,Log Entry Type,Controller Output,Reference Error,Integrated Error
Log Entry,,,,,,
1,501,0.501,CONTROLLER_OUTPUT,0.0,0.0,0.0
16,513,0.513,CONTROLLER_OUTPUT,0.0,0.0,0.0
24,523,0.523,CONTROLLER_OUTPUT,0.0,0.0,0.0
37,534,0.534,CONTROLLER_OUTPUT,0.0,0.0,0.0
45,545,0.545,CONTROLLER_OUTPUT,0.0,0.0,0.0


In [50]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
controller_out_log.to_csv(f'{split_logs_dir}/controller_out.csv')

# Message Log

In [51]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MOTOR'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,
160,686,MOTOR,"-2,1579197861",0.686
202,736,MOTOR,"-2,0",0.736
243,786,MOTOR,"-2,0",0.786
285,836,MOTOR,"-2,0",0.836
326,886,MOTOR,"-2,0",0.886


In [52]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/motor.csv')

In [53]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MSG'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,


In [54]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/messages.csv')